**1st Cycle Tech Support Team Work Summary**

**This script is mainly used to grab tables in PDF and generate CSV files  
script doesn't fit well before 2000  
(Jia Pengyue&emsp;&emsp;2020.6.24)**  

## 导入所需依赖

这里要pip install PyMuPDF 安装fitz的辅助环境  
其余包pip install即可

> pip install fitz lxml==4.3 networkx==2.0 -i https://pypi.tuna.tsinghua.edu.cn/simple/

In [5]:
import os
from pathlib2 import Path
import pandas as pd
import fitz
import glob
import tabula
import pytesseract
import subprocess
from PIL import Image

## 创建文件目录及声明路径

首先创建文件目录，如下图所示。  
pdf文件存于pdfs文件夹中

|---geology_big_data  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|---pdfs  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|---figures  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|---ABBYY  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|---result_ZJUgeology  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|---csv

In [3]:
# 整个项目的根目录
root_path = 'C:\\Users\\Administrator\\Desktop\\code\\python\\GIS-python\\geology_big_data\\'
pdf_path = root_path + 'pdfs\\'
#result文件夹的命名请尽量复杂，确保不会在论文名称中出现
result_path = root_path + 'result_ZJUgeology\\'
figure_path = root_path + 'figures\\'
ABBYY_path = root_path + 'ABBYY\\'

## 抓取表格另存为png图片

1.需要提前安装PDFFigures2  
https://github.com/allenai/pdffigures2  
https://titipata.github.io/2016/11/06/pdffigure2.html  
2.安装java环境（注意配置完环境变量）  
https://www.runoob.com/java/java-environment-setup.html#win-install  
3.安装sbt  
注意这里sbt的安装路径不要有空格  
https://www.scala-sbt.org/download.html  
安装的文档  
https://blog.csdn.net/weixin_41846320/article/details/84784175   
使用说明  
https://titipata.github.io/2016/11/06/pdffigure2.html  
> **2、3两步安装好的测试都是在cmd内输入java -version 与 sbt -version 如果有显示，则说明安装完毕了**

出现Java的不能运行powershell.exe错误，则需为系统加入环境变量C:\Windows\System32\WindowsPowerShell\v1.0

### 创建result文件夹

In [9]:
files = os.listdir(path=pdf_path)
for pdf in files:
    pdf_name = pdf[:-4] #去除.pdf后缀
    #在结果文件夹中创建pdf名字的文件夹
    subprocess.run(["cmd","/c","mkdir","{}{}".format(result_path, pdf_name)],shell=True)

### 创建figures对应文件夹

In [10]:
for pdf in files:
    pdf_name = pdf[:-4] #去除.pdf后缀
    os.system('mkdir {}{}'.format(figure_path,pdf_name))

### 提取图片和表格存为png

In [11]:
#将目录调整至pdffigures2文件夹
os.chdir('C:\\Users\\Administrator\\pdffigures2')
#执行sbt语句
for pdf in files:
    pdf_name = pdf[:-4]
    os.system('sbt "runMain org.allenai.pdffigures2.FigureExtractorBatchCli {0}{1} -s {2}.json -m {3}{4}\\{5} -d {6}{7}\\{8}"'\
          .format(pdf_path,pdf,pdf_name,result_path,pdf_name,pdf_name,result_path,pdf_name,pdf_name))

## 生成表格PDF

### 筛选表格png

In [12]:
for root,dirnames,files in os.walk('C:/Users/Administrator/Desktop/code/python/GIS-python/geology_big_data/result_ZJUgeology'):
    for file in files:
        if 'Table' not in file:
            os.system('move {0}\\{1} {2}'.format(root,file,root.replace('result_ZJUgeology','figures')))

### 创建同名pdf文件

In [13]:
for root,dirnames,files in os.walk('C:/Users/Administrator/Desktop/code/python/GIS-python/geology_big_data/result_ZJUgeology'):
    for file in files:
        if '.png' in file:
            path_file = '{}\{}'.format(root,file)
            path_file = path_file.replace('/','\\')
            imgdoc = fitz.open(path_file)
            pdfbytes = imgdoc.convertToPDF()    # 使用图片创建单页的 PDF
            imgpdf = fitz.open("pdf", pdfbytes)
            imgpdf.save(path_file[:-4] + '.pdf')

## 用pytesseract转换png为pdf

需要提前配置tesseract

In [14]:
for root,dirnames,files in os.walk('C:/Users/Administrator/Desktop/code/python/GIS-python/geology_big_data/result_ZJUgeology'):
    for file in files:
        if '.png' in file:
            pdf = pytesseract.image_to_pdf_or_hocr("{}\\{}".format(root,file), extension='pdf')
            with open('{}\\{}.pdf'.format(root,file[:-4]), 'w+b') as f:
                f.write(pdf) # pdf type is bytes by default

## 使用tabula提取表格并保存为csv

In [15]:
for root,dirnames,files in os.walk('C:/Users/Administrator/Desktop/code/python/GIS-python/geology_big_data/result_ZJUgeology'):
    for file in files:
        if '.pdf' in file:
            try:
                tabula.io.convert_into('{}\\{}'.format(root,file),"C:\\Users\\Administrator\\Desktop\\code\\python\\GIS-python\\geology_big_data\\csv\\{}.csv".format(file[:-4]),output_format='csv',pages='all')
            except Exception:
                pass